In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
df= pd.read_csv('data/Transactions.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
isfraud = df.groupby(["type", "isFraud", "isFlaggedFraud"])["type"].count()
isfraud_df = isfraud.to_frame(name='count').reset_index()
isfraud_df.columns = ["type", "Actual Fraud", "Flagged Fraud", "Count"]
isfraud_df["Actual Fraud"] = isfraud_df["Actual Fraud"].apply(lambda x: "Yes" if x==1 else "No")
isfraud_df["Flagged Fraud"] = isfraud_df["Flagged Fraud"].apply(lambda x: "Yes" if x==1 else "No")
isfraud_df

,type,Actual Fraud,Flagged Fraud,Count
0,CASH_IN,No,No,1399284
1,CASH_OUT,No,No,2233384
2,CASH_OUT,Yes,No,4116
3,DEBIT,No,No,41432
4,PAYMENT,No,No,2151495
5,TRANSFER,No,No,528812
6,TRANSFER,Yes,No,4081
7,TRANSFER,Yes,Yes,16


#### Distribution of transactions by type

In [25]:
type_counts = isfraud_df.groupby("type")["Count"].sum().reset_index()
type_counts.columns = ["Type", "Total Count"]

# --- Plotly Express Plot Generation ---
fig = px.bar(
    type_counts,
    x='Type',
    y='Total Count',
    title='Distribution of Transaction Type by Count',
    labels={'Total Count': 'Count', 'Type': 'Transaction Type'},
    color='Type', # Assigns a different color to each bar
    color_discrete_sequence=['#004d99'] # Use a single color if preferred
)

# Customize layout
fig.update_layout(
    xaxis_title='Transaction Type',
    yaxis_title='Count',
    xaxis={'categoryorder': 'total descending'} # Orders bars by count (optional)
)

# fig.show()

#### Distribution of transaction types that where targeted.

In [ ]:
actual_fraud_counts = isfraud_df[isfraud_df["Actual Fraud"] == "Yes"].groupby("type")["Count"].sum()
flagged_fraud_counts = isfraud_df[isfraud_df["Flagged Fraud"] == "Yes"].groupby("type")["Count"].sum()

combined_df = pd.DataFrame({
    'Actual Fraud': actual_fraud_counts,
    'Flagged Fraud': flagged_fraud_counts
}).fillna(0).reset_index()

plot_df = combined_df.melt(
    id_vars='type',
    value_vars=['Actual Fraud', 'Flagged Fraud'],
    var_name='Category',
    value_name='Count'
)

fig = px.bar(
    plot_df,
    x='type',
    y='Count',
    color='Category',
    barmode='group',
    title='Distribution of Transaction Type by Fraud Count Status',
    labels={
        'type': 'Transaction Type',
        'Count': 'Count',
        'Category': 'Fraud Status'
    },
    color_discrete_map={
        'Actual Fraud': '#004d99',
        'Flagged Fraud': '#ff6b6b'
    }
)

fig.update_traces(
    texttemplate='%{y}',
    textposition='outside'
)


fig.update_layout(
    uniformtext_minsize=8, 
    uniformtext_mode='hide',
    xaxis_title='Transaction Type',
    yaxis_title='Count',
    legend_title='Fraud Status',
    hovermode='x unified'
)


#### Confusion matrix to identify True positives and False Negatives.

In [23]:
df = df.rename(columns = {"isFraud": "Actual Fraud", "isFlaggedFraud": "Detected Fraud"})

cm = confusion_matrix(df["Actual Fraud"], df["Detected Fraud"])
display_labels = ['No Fraud', 'Fraud']

# --- Plotly Heatmap Generation ---
fig = px.imshow(
    cm,
    text_auto=True,  
    x=display_labels, 
    y=display_labels, 
    color_continuous_scale='Blues',
    title='Confusion Matrix - Fraud Detection'
)

# Customizing the layout
fig.update_xaxes(side="top", title_text='Predicted Label')
fig.update_yaxes(title_text='True Label')
fig.update_layout(
    coloraxis_showscale=False # Hides the color legend as the numbers are displayed
)

# fig.show()